In [1]:
from constants import FLUXES_FOLDER, \
                      SLM_FOLDER

from data_utils import process_amp_phase_data, \
                       add_row_padding

from plot_utils import plot_map, \
                       plot_model_history, \
                       plot_autoencoder

from modeling_utils import create_autoencoder_for_flux, \
                           compile_model, \
                           train_model, \
                           store_model

from configurations import AutoEncoderConfiguration, \
                           EncoderConvolutionalConfiguration

2023-12-04 09:06:16.507722: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-04 09:06:17.195856: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
fluxes_filename = "all_fluxes.npy"
amplitudes_filename = "complexsine_pupamp.npy"
phases_filename = "complexsine_pupphase.npy"

fluxes_path = f"{FLUXES_FOLDER}/{fluxes_filename}"
amplitudes_path = f"{SLM_FOLDER}/{amplitudes_filename}"
phases_path = f"{SLM_FOLDER}/{phases_filename}"

In [3]:
train_fluxes_array, val_fluxes_array, train_amp_phase_array, val_amp_phase_array, scalers = \
    process_amp_phase_data(
        fluxes_path, 
        amplitudes_path,
        phases_path,
        n_points=10000,
        trim_amplitude=True,
        trim_phase=True,
        normalize_flux=True,
        normalize_amplitude=True,
        shuffle=False,
        flatten_fluxes=False,
        split=True,
        val_ratio=0.1
    )

In [4]:
train_fluxes_array = add_row_padding(train_fluxes_array, 1)
val_fluxes_array = add_row_padding(val_fluxes_array, 1)

In [5]:
model_configuration = AutoEncoderConfiguration(train_fluxes_array)
print(model_configuration.get_description())


	=== FluxAutoencoder ===
	*ARCHITECTURE HYPERPARAMETERS:
		-Autoencoder
		-Input shape: (56, 24)
		-Convolutional Layers: [256, 128, 16, 4] (Inverse in the decoder)
		-Convolutonal Kernels: [(3, 3), (3, 3), (3, 3), (3, 3)] (Inverse in the decoder)
		-Convolutional Activation: relu
		-Output Layer Activation: linear
	
	*COMPILATION HYPERPARAMETERS:
		-Optimizer: ADAM lr=0.001, beta_1=0.9, beta_2=0.999
		-Loss Function: MSE
		-Metric: MSE
	
	* TRAINING HYPERPARAMETERS:
		-Epochs: 1
		-Batch size: 16
		-Callbacks:
			-ReduceLROnPlateau: MSE 8 x0.1
			-Early Stop: MSE 15
	


2023-12-04 09:06:21.631284: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-04 09:06:21.670808: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-04 09:06:21.671094: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [6]:
model = create_autoencoder_for_flux(
    *model_configuration.unpack_architecture_hyperparameters()
)


Model: "FluxAutoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 56, 24, 256)       2560      
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 24, 256)       590080    
                                                                 
 max_pooling2d (MaxPooling2  (None, 28, 12, 256)       0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 12, 128)       295040    
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 12, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 6, 128)        0         
 g2D)                                              

In [7]:
compile_model(
    model,
    *model_configuration.unpack_compilation_hyperparameters()
)

In [ ]:
history = train_model(
    model,
    train_fluxes_array,
    train_fluxes_array,
    val_fluxes_array,
    val_fluxes_array,
    *model_configuration.unpack_training_hyperparameters()
)

2023-12-04 09:06:25.103869: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-12-04 09:06:25.883519: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.21GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-04 09:06:25.883564: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.21GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-04 09:06:26.692963: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7ff1b112c540 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-04 09:06:26.693004: I tensorflow/compiler/xla/service/service.cc:176]   Str

561/563 [============================>.] - ETA: 0s - loss: 0.2688 - mean_squared_error: 0.2688

In [ ]:
plot_model_history(history)

In [ ]:
plot_autoencoder(model, val_fluxes_array[757])

In [ ]:
plot_autoencoder(model, val_fluxes_array[758])

In [ ]:
store_model(model,
            model.name,
            model_configuration.get_description())

In [ ]:
for layer in encoder.layers:
    layer.trainable = False
    if layer.name == "bottleneck":
        break
    

In [ ]:
encoder.summary()

In [ ]:
conv_input = encoder.output
conv_layers = keras.layers.UpSampling2D(size=(1,2))(conv_input)
conv_layers = keras.layers.UpSampling2D(size=(2,2))(conv_layers)
conv_layers = keras.layers.UpSampling2D(size=(2,2))(conv_layers)
conv_layers = keras.layers.UpSampling2D(size=(2,2))(conv_layers)
conv_layers = keras.layers.UpSampling2D(size=(2,2))(conv_layers)

conv_model = keras.Model(inputs=encoder.input, outputs=conv_layers)
conv_model.summary()

In [ ]:
model_configuration = EncoderConvolutionalConfiguration()